In [ ]:
print("all_ok")

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
model=ChatGroq(model="qwen/qwen3-32b")

In [ ]:
model.invoke("What is hte capital of America?").content

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
embedding_model.embed_query("What is hte capital of America?")

## 1. Data Ingestion

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
file_path = os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
loader = PyPDFLoader(file_path)

In [ ]:
documents = loader.load()

In [ ]:
documents

In [ ]:
len(documents)  

## Chuncking using RecursiveCharacterTextSplitter

## this is a experimental thing there is no deterministic way to split the text

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

### in the above chunk_size = 500 means 500 characters per chunk

In [ ]:
docs=text_splitter.split_documents(documents)

In [ ]:
len(docs)

## command to get metadata of each document

In [ ]:
docs[0].metadata

## command to get page content of a document

In [ ]:
docs[0].page_content

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
vectorstore=FAISS.from_documents(docs, embedding_model)

### Above vectorstore is a in memory store
Types of Vector Stores 
1) In Memory (faiss is in memory vector store, chroma)
2) On Disk Storage (faiss you can persist over the disck, chroma)
3) Cloud Storage (cloud variant of faiss is not available for cloud we can use Pinecone, Weaviate, MangoDBVectorSearch, AstraDb etc.,)

##Below line is to perform similarity search and this process is called Retrieval Process. Means from the vectordatabase we are going to fetch or retrieve or rank the most appropriate k result

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.")

In [ ]:
relevant_doc

In [ ]:
relevant_doc[0].page_content

In [ ]:
relevant_doc[1].page_content

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=10)

In [ ]:
relevant_doc

# Generation Process

In [91]:
retriever = vectorstore.as_retriever()

In [92]:
retriever.invoke("llama2 finetuning benchmark experiments.",k=10)

[Document(id='5f4b08f9-f937-4c5b-b3c2-49796144e5de', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/satishpolasi/Documents/LLMOPS/document_portal/notebook/data/sample.pdf', 'total_pages': 77, 'page': 7, 'page_label': '8'}, page_content='13B 18.9 66.1 52.6 62.3 10.9 46.9 37.0 33.9\n33B 26.0 70.0 58.4 67.6 21.4 57.8 39.8 41.7\n65B 30.7 70.7 60.5 68.6 30.8 63.4 43.5 47.6\nLlama 2\n7B 16.8 63.9 48.9 61.3 14.6 45.3 32.6 29.3\n13B 24.5 66.9 55.4 65.8 28.7 54.8 39.4 39.1\n34B 27.8 69.9 58.7 68.0 24.2 62.6 44.1 43.4\n70B 37.5 71.9 63.6 69.4 35.2 68.9 51.2 54.2\nTable 3: Overall performance on grouped academic benchmarks compared to open-source base models.'),
 Document(id='633dab6a

### Question: User question
### Context: based on the question retrieving the info from the vector database

In [93]:
prompt_template = """
Answer the question based on the context provided below.
If the context does not contain sufficient information, respond with:
'I do not have enough information about this."

Context: {context}

Question: {question}

Answer:"""

In [94]:
from langchain.prompts import PromptTemplate

In [95]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)

In [96]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context provided below.\nIf the context does not contain sufficient information, respond with:\n\'I do not have enough information about this."\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:')

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context provided below.\nIf the context does not contain sufficient information, respond with:\n\'I do not have enough information about this."\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:')

In [99]:
from langchain_core.output_parsers import StrOutputParser

In [100]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [104]:
from langchain_core.runnables import RunnablePassthrough

In [105]:
rag_chain= (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [106]:
rag_chain.invoke("tell me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s see. The user is asking about the Llama2 fine-tuning benchmark experiments. I need to look at the context provided to find relevant information. \n\nLooking at the context, there\'s a section labeled "Fine-tuned" under Llama 2. The numbers there might represent the benchmark results. The context includes different model sizes like 7B, 13B, 34B, 70B for Llama 2. Each has a series of numbers. But the user specifically mentioned "finetuning benchmark experiments." \n\nIn the context, after the tables, there\'s a mention of "Fine-tuned" and "ChatGPT" with some numbers. The Llama 2 fine-tuned section has entries like 7B, 13B, 34B, 70B with numbers following them. However, the numbers are a bit jumbled. For example, under Llama 2\'s Fine-tuned section, the 7B line has numbers like 0.28, 0.25, etc., and the 13B line has 0.24, 0.25, 0.35... but then there\'s a line that says "0.50 0.36..." which might be part of the same model\'s results. The formatting here is a bit unc